<a href="https://colab.research.google.com/github/gopichand909/data-science/blob/main/Toddler_Cry_Reason.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/warcoder/infant-cry-audio-corpus")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sillycelebrity
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/warcoder/infant-cry-audio-corpus


100%|██████████| 39.0M/39.0M [00:00<00:00, 83.7MB/s]


In [55]:
import pandas as pd


data = pd.read_csv('/content/donateacrycorpusfeaturesdataset/donateacry-corpus_features_final.csv')


cry_reason_3 = data[data['Cry_Reason'] == 3]


cry_reason_3_sampled = cry_reason_3.sample(n=50, random_state=42)


data_reduced = pd.concat([cry_reason_3_sampled, data[data['Cry_Reason'] != 3]])

data_reduced = data_reduced.sample(frac=1, random_state=42).reset_index(drop=True)

data_reduced.to_csv('/content/donateacrycorpusfeaturesdataset/donateacry-corpus_features_reduced.csv', index=False)



In [65]:
import pandas as pd
data = pd.read_csv('/content/donateacrycorpusfeaturesdataset/donateacry-corpus_features_reduced.csv')
data.head()

,Cry_Audio_File,Cry_Reason,Amplitude_Envelope_Mean,RMS_Mean,ZCR_Mean,STFT_Mean,SC_Mean,SBAN_Mean,SCON_Mean,MFCCs13Mean,...,MFCCs4,MFCCs5,MFCCs6,MFCCs7,MFCCs8,MFCCs9,MFCCs10,MFCCs11,MFCCs12,MFCCs13
0,/Users/mac/Desktop/Smart_Cradle/donateacry_cor...,3,0.084653,0.034276,0.071341,1.578388,918.886879,399.735475,24.874112,-34.173640,...,18.009497,23.897955,-8.260139,0.644967,-0.934566,-14.655108,-3.018429,-4.830470,-15.399602,-7.640189
1,/Users/mac/Desktop/Smart_Cradle/donateacry_cor...,3,0.045999,0.024169,0.068564,0.584260,1070.948304,651.746237,28.254034,-44.522007,...,-12.725027,-0.810158,-24.505787,-10.729454,-0.127280,-13.929709,-13.525310,-8.579855,-6.817992,-0.133577
2,/Users/mac/Desktop/Smart_Cradle/donateacry_cor...,3,0.045073,0.020738,0.093485,0.498600,1178.619167,552.890775,27.714983,-42.699688,...,-10.539698,5.293349,-22.207100,-7.739832,3.716520,-7.233214,-3.811746,-7.538359,-13.982996,-1.217895
3,/Users/mac/Desktop/Smart_Cradle/donateacry_cor...,2,0.081248,0.026975,0.115286,0.597292,1397.014843,663.220789,26.767674,-37.134360,...,-1.846886,11.767962,-31.118536,4.112152,8.710549,-18.142767,1.907115,1.202223,-12.527912,5.186280
4,/Users/mac/Desktop/Smart_Cradle/donateacry_cor...,0,0.027163,0.010520,0.085081,0.140766,1089.415328,668.895951,25.608310,-44.275997,...,-1.704598,0.226281,-29.814095,-14.937744,-11.162116,-21.503994,-7.501309,-7.856627,-15.119538,-3.970483


In [66]:
import pandas as pd
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('/content/donateacrycorpusfeaturesdataset/donateacry-corpus_features_reduced.csv')

data = data.dropna()

# Prepare the features (X) and target (y)
feature_columns = [
    'Amplitude_Envelope_Mean', 'RMS_Mean', 'ZCR_Mean', 'STFT_Mean',
    'SC_Mean', 'SBAN_Mean', 'SCON_Mean', 'MFCCs1', 'MFCCs2', 'MFCCs3',
    'MFCCs4', 'MFCCs5', 'MFCCs6', 'MFCCs7', 'MFCCs8', 'MFCCs9',
    'MFCCs10', 'MFCCs11', 'MFCCs12', 'MFCCs13'
]
X = data[feature_columns]
y = data['Cry_Reason']

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)


cry_reason_mapping = {
    0: 'belly pain',
    1: 'burping',
    2: 'discomfort',
    3: 'hungry',
    4: 'tired'
}

# Function to extract features from an audio file
def extract_features(audio_file_path):
    y, sr = librosa.load(audio_file_path, sr=None)

    try:
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, fmin=50.0, n_bands=4)
        spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
    except librosa.util.exceptions.ParameterError as e:
        return f"Error in spectral contrast computation: {e}"

    features = {
        'Amplitude_Envelope_Mean': np.mean(np.abs(y)),
        'RMS_Mean': np.mean(librosa.feature.rms(y=y)),
        'ZCR_Mean': np.mean(librosa.feature.zero_crossing_rate(y)),
        'STFT_Mean': np.mean(np.abs(librosa.stft(y))),
        'SC_Mean': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        'SBAN_Mean': np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
        'SCON_Mean': spectral_contrast_mean[0] if len(spectral_contrast_mean) > 0 else np.nan,
    }

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(1, 14):
        features[f'MFCCs{i}'] = np.mean(mfccs[i-1])

    return pd.DataFrame([features])

def predict_cry_reason(audio_file_path):
    try:

        features_df = extract_features(audio_file_path)
        features_df = features_df[feature_columns]
        prediction = model.predict(features_df)
        cry_reason_code = prediction[0]
        cry_reason = cry_reason_mapping.get(cry_reason_code, 'Unknown')
        return cry_reason
    except Exception as e:
        return f"Error: {e}"


audio_file_path = input("Enter path :")
predicted_reason = predict_cry_reason(audio_file_path)
print(f"Predicted Cry Reason: {predicted_reason}")



Enter path :/content/infant-cry-audio-corpus/donateacry_corpus/belly_pain/643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138495-1.0-m-72-bp.wav
Predicted Cry Reason: belly pain
